<a href="https://colab.research.google.com/github/JorgeAnsotegui/TFM/blob/main/Dataset_Tweaking/Rename_Labels%26Images.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Conectar Colab a Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install ruamel.yaml

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.8/117.8 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.7/526.7 kB 6.5 MB/s eta 0:00:00


In [8]:
import zipfile
import os
import yaml
from ruamel.yaml import YAML
import re
import shutil

# Ruta del archivo zip en Google Drive
ruta_zip = '/content/drive/MyDrive/TFM/Sin duplicados/Polipos Final Images.v2-imageleveldataaugmentation.yolov9.zip'

# Ruta de la carpeta destino
ruta_destino = '/content/dataset_2'

# Crear la carpeta destino si no existe
if not os.path.exists(ruta_destino):
    os.makedirs(ruta_destino)

# Descomprimir el archivo zip
with zipfile.ZipFile(ruta_zip, 'r') as zip_ref:
    zip_ref.extractall(ruta_destino)

print(f'Archivo descomprimido en {ruta_destino}')

# Ruta vieja y nueva
ruta_valid = os.path.join(ruta_destino, 'valid')
ruta_val = os.path.join(ruta_destino, 'val')

# Ruta del archivo data.yaml
ruta_yaml = os.path.join(ruta_destino, 'data.yaml')

print(f'Archivo descomprimido en {ruta_destino}')

# Verificar si existe la carpeta 'valid' y cambiar su nombre a 'val'
if os.path.exists(ruta_valid):
    os.rename(ruta_valid, ruta_val)
    print(f"Carpeta 'valid' renombrada a 'val'.")

    # Ruta del archivo data.yaml
    ruta_yaml = os.path.join(ruta_destino, 'data.yaml')

    # Modificar el archivo data.yaml
    if os.path.exists(ruta_yaml):
        yaml = YAML()
        with open(ruta_yaml, 'r') as file:
            data = yaml.load(file)

        # Cambiar 'valid' por 'val' en las rutas del archivo
        if 'val' in data and data['val'] == '../valid/images':
            data['val'] = '../val/images'

        with open(ruta_yaml, 'w') as file:
            yaml.dump(data, file)

        print(f"Archivo 'data.yaml' modificado correctamente.")
    else:
        print("El archivo 'data.yaml' no existe.")
else:
    print("La carpeta 'valid' no existe.")

Archivo descomprimido en /content/dataset_2
Archivo descomprimido en /content/dataset_2
Carpeta 'valid' renombrada a 'val'.
Archivo 'data.yaml' modificado correctamente.


# Falta que funcione para todas las carpetas no solo para train


In [ ]:
# Función para renombrar archivos
def renombrar_archivos(carpeta, extension):
    for nombre_archivo in os.listdir(carpeta):
        if nombre_archivo.endswith(extension):
            nuevo_nombre = re.sub(r'_jpg\.rf\.[a-f0-9]{32}', '', nombre_archivo)
            ruta_actual = os.path.join(carpeta, nombre_archivo)
            nueva_ruta = os.path.join(carpeta, nuevo_nombre)
            os.rename(ruta_actual, nueva_ruta)
            print(f'Renombrado {nombre_archivo} a {nuevo_nombre}')

# Rutas para el conjunto de entrenamiento
carpeta_etiquetas_train = os.path.join(ruta_destino, 'train/labels')
carpeta_imagenes_train = os.path.join(ruta_destino, 'train/images')

# Rutas para el conjunto de validación
carpeta_etiquetas_val = os.path.join(ruta_destino, 'val/labels')
carpeta_imagenes_val = os.path.join(ruta_destino, 'val/images')

# Rutas para el conjunto de prueba
carpeta_etiquetas_test = os.path.join(ruta_destino, 'test/labels')
carpeta_imagenes_test = os.path.join(ruta_destino, 'test/images')

# Renombrar archivos en las carpetas de etiquetas y de imágenes
renombrar_archivos(carpeta_etiquetas_train, ".txt")
renombrar_archivos(carpeta_imagenes_train, ".jpg")

renombrar_archivos(carpeta_etiquetas_val, ".txt")
renombrar_archivos(carpeta_imagenes_val, ".jpg")

renombrar_archivos(carpeta_etiquetas_test, ".txt")
renombrar_archivos(carpeta_imagenes_test, ".jpg")


# Intento de no sobreescribir imagenes


In [9]:
import os
import re

# Función para renombrar archivos
def renombrar_archivos(carpeta, extension, DataAugmentation):
    for nombre_archivo in os.listdir(carpeta):
        if nombre_archivo.endswith(extension):
            # Obtener el nuevo nombre base sin el patrón '_jpg.rf.[hash]'
            nuevo_nombre_base = re.sub(r'_jpg\.rf\.[a-f0-9]{32}', '', nombre_archivo)
            nuevo_nombre, ext = os.path.splitext(nuevo_nombre_base)

            ruta_actual = os.path.join(carpeta, nombre_archivo)

            if DataAugmentation:
                contador = 1
                nuevo_nombre_con_numero = f"{nuevo_nombre}_{contador}{ext}"
                nueva_ruta = os.path.join(carpeta, nuevo_nombre_con_numero)

                # Verificar si ya existe un archivo con el nombre deseado y ajustar el contador
                while os.path.exists(nueva_ruta):
                    contador += 1
                    nuevo_nombre_con_numero = f"{nuevo_nombre}_{contador}{ext}"
                    nueva_ruta = os.path.join(carpeta, nuevo_nombre_con_numero)
            else:
                nueva_ruta = os.path.join(carpeta, nuevo_nombre_base)

            os.rename(ruta_actual, nueva_ruta)
            print(f'Renombrado {nombre_archivo} a {nuevo_nombre_con_numero if DataAugmentation else nuevo_nombre_base}')

# Rutas para el conjunto de entrenamiento
carpeta_etiquetas_train = os.path.join(ruta_destino, 'train/labels')
carpeta_imagenes_train = os.path.join(ruta_destino, 'train/images')

# Rutas para el conjunto de validación
carpeta_etiquetas_val = os.path.join(ruta_destino, 'val/labels')
carpeta_imagenes_val = os.path.join(ruta_destino, 'val/images')

# Rutas para el conjunto de prueba
carpeta_etiquetas_test = os.path.join(ruta_destino, 'test/labels')
carpeta_imagenes_test = os.path.join(ruta_destino, 'test/images')

# Renombrar archivos en las carpetas de etiquetas y de imágenes
renombrar_archivos(carpeta_etiquetas_train, ".txt", DataAugmentation=True)
renombrar_archivos(carpeta_imagenes_train, ".jpg", DataAugmentation=True)

renombrar_archivos(carpeta_etiquetas_val, ".txt", DataAugmentation=False)
renombrar_archivos(carpeta_imagenes_val, ".jpg", DataAugmentation=False)

renombrar_archivos(carpeta_etiquetas_test, ".txt", DataAugmentation=False)
renombrar_archivos(carpeta_imagenes_test, ".jpg", DataAugmentation=False)


Renombrado 100H0034_jpg.rf.22c77519130f079a16695b64905ec1af.txt a 100H0034_1.txt
Renombrado 100H0242_jpg.rf.6e15c9aebd3ee86354390e152990d96b.txt a 100H0242_1.txt
Renombrado 100H0238_jpg.rf.875f56f3232ec567f32fbc464e2955a7.txt a 100H0238_1.txt
Renombrado 100H0075_jpg.rf.acf2b9fb85ae514d1a27995457fe741f.txt a 100H0075_1.txt
Renombrado 100H0032_jpg.rf.37cd49a00d027ab23d2fda2f733cce98.txt a 100H0032_1.txt
Renombrado 100H0063_jpg.rf.8a52d4d45c0f575658a55732863b496b.txt a 100H0063_1.txt
Renombrado 100H0177_jpg.rf.3cdea358844cb87fe27284a0a3a15bdb.txt a 100H0177_1.txt
Renombrado 100H0252_jpg.rf.d9fc7d030088fcf666e1fbc87b8808bb.txt a 100H0252_1.txt
Renombrado 100H0184_jpg.rf.16986d7b5fb12fd3e44015e48733625f.txt a 100H0184_1.txt
Renombrado 100H0201_jpg.rf.2bf0992c6a5f08e19f70f85c6b24be1c.txt a 100H0201_1.txt
Renombrado 100H0175_jpg.rf.20152519501c8370957f94009a53bbc0.txt a 100H0175_1.txt
Renombrado 100H0116_jpg.rf.9c02de7ed587be053173d748b9344182.txt a 100H0116_1.txt
Renombrado 100H0177_jpg.rf.c

#Muevo los resultados a mi drive

In [ ]:
import shutil
# Directorio de origen
src_dir = ruta_destino

# Directorio de destino
dest_dir = "/content/drive/MyDrive/TFM/Sin duplicados/Bounding_Box_Data_Augmentation"

def copy_directory(src, dest):
    if not os.path.exists(dest):
        os.makedirs(dest)

    for root, dirs, files in os.walk(src):
        for name in dirs:
            src_dir = os.path.join(root, name)
            dest_dir = os.path.join(dest, os.path.relpath(src_dir, src))
            if not os.path.exists(dest_dir):
                os.makedirs(dest_dir)

        for name in files:
            src_file = os.path.join(root, name)
            dest_file = os.path.join(dest, os.path.relpath(src_file, src))
            if not os.path.exists(os.path.dirname(dest_file)):
                os.makedirs(os.path.dirname(dest_file))
            shutil.copy2(src_file, dest_file)

# Copiar el directorio de origen al directorio de destino
copy_directory(src_dir, dest_dir)

print(f"Directorio copiado de {src_dir} a {dest_dir}")

Directorio copiado de /content/dataset_4 a /content/drive/MyDrive/TFM/Sin duplicados/Bounding_Box_Data_Augmentation


In [ ]:
import os

# Ruta de la carpeta
carpeta_imagenes = '/content/drive/MyDrive/TFM/Sin duplicados/Bounding_Box_Data_Augmentation/train/images'

# Contar archivos .jpg
num_jpg = len([archivo for archivo in os.listdir(carpeta_imagenes) if archivo.endswith('.jpg')])

print(f'Número de archivos .jpg en la carpeta: {num_jpg}')


Número de archivos .jpg en la carpeta: 139
